In [12]:
from flask import Flask , request

class ML:
    def __init__(self):
        self.avaliable_models = {
            "face_detection": "/additional_drive/ML/face_detection",
            "car_detection": "/additional_drive/ML/car_detection",
            "shoe_detection": "/additional_drive/ML/shoe_detection",
            "cloth_detection": "/additional_drive/ML/cloth_detection",
            "signal_detection": "/additional_drive/ML/signal_detection",
            "water_level_detection": "/additional_drive/ML/water_level_detection",
            "missile_detection": "/additional_drive/ML/missile_detection"
        }
        self.loaded_models_limit = 2
        self.loaded_models = {
            model: {"path": self.load_weights(model), "requests": 0}
            for model in list(self.avaliable_models)[:self.loaded_models_limit]
        }

    def load_weights(self, model):
        return self.avaliable_models.get(model,None)

    def load_balancer(self, new_model):
        model_requests = [(model, data["requests"]) for model, data in self.loaded_models.items()]
        least_used_model, least_used_model_requests = min(model_requests, key=lambda x: x[1])
        self.loaded_models.pop(least_used_model)
        self.loaded_models[new_model] = {"path": self.load_weights(new_model), "requests": least_used_model_requests}
        
    def increment_requests(self, model):
        self.loaded_models[model]["requests"] += 1


app = Flask(__name__)
ml = ML()

@app.route('/get_loaded_models', methods=['GET', 'POST'])
def get_loaded_models():
    return ml.loaded_models

@app.route('/process_request', methods=['GET', 'POST'])
def process_request():
    try:
        model = request.form["model"]
        if model not in ml.loaded_models:
            ml.load_balancer(model)
        ml.increment_requests(model)
        return "processed by " + ml.loaded_models[model]["path"]
    except:
        return str(traceback.format_exc())

app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.10.50:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Apr/2023 15:27:00] "GET /get_loaded_models HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "POST /process_request HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "GET /get_loaded_models HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "POST /process_request HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "GET /get_loaded_models HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "POST /process_request HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "GET /get_loaded_models HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "POST /process_request HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "GET /get_loaded_models HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "POST /process_request HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 15:27:00] "GET /get_loaded_models HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2